<a href="https://colab.research.google.com/github/pas3n7/ColabDataScienceLearning/blob/main/pytorchmnist_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=i2yPxY2rOzs

In [128]:
import torch
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import numpy as np

In [129]:
train = datasets.MNIST("", train=True, download=True,
                       transform = transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("", train=False, download=True,
                       transform = transforms.Compose([transforms.ToTensor()]))

In [130]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=True)

In [178]:
def showgrid(atensor, gridside=4):
    plt.figure(figsize=(8, 8))
    plt.imshow(np.concatenate([atensor.dataset.data[i:i+gridside].view((28*gridside, 28)) for i in range(0, gridside**2, gridside)], axis=1))
    plt.show()

In [133]:
for data in trainset:
    print(data)
    break

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0

In [174]:
total = 0
counter_dict = {i:0 for i in range(10)}

for data in trainset:
    Xs, ys = data
    for y in ys:
        counter_dict[int(y)] += 1
        total+=1

for i in counter_dict:
    print(f"{i}: {counter_dict[i]/total*100:2f}")

0: 9.871667
1: 11.236667
2: 9.930000
3: 10.218333
4: 9.736667
5: 9.035000
6: 9.863333
7: 10.441667
8: 9.751667
9: 9.915000


In [175]:
import torch.nn as nn
import torch.nn.functional as F

In [177]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64) #1st fully connected layer , images are 28x28
        self.fc2 = nn.Linear(64, 64) #hidden layer
        self.fc3 = nn.Linear(64, 64) #hidden layer
        self.fc4 = nn.Linear(64, 10) #output layer

    def forward(self, x):
        #input is x
        x = self.fc1(x) #x passes through fc1, we redefine x then...
        x = F.relu(x) #activation function
        x = self.fc2(x) #then passes through fc2
        x = F.relu(x)
        x = self.fc3(x) #etc
        x = F.relu(x)
        x = self.fc4(x) #etc, relu doesn't make sense for output
        #for output we want a probability distribution, because we have multiple classes to match
        return F.log_softmax(x, dim=1) #dim 1, will be flat


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)
